# Custom segmentation Vizgen data

We here provide commands to segment Vizgen data into the different segmentations. Raw data can be accessed via Vizgen. If data is not available there anymore, raw data can be requested with the corresponding authors.
This notebook is not intended to be executed but rather is a collection of batch commands to execute segmentation and provides the commands used by us.

## Read all format Vizgen to spatialdata

In [1]:
import spatialdata_io

/home/cane/miniconda3/envs/resolvi_new/lib/python3.10/site-packages/cudf/utils/_numba.py:110: UserWarning: Using CUDA toolkit version (12, 5) with CUDA driver version (12, 4) requires minor version compatibility, which is not yet supported for CUDA driver versions 12.0 and above. It is likely that many cuDF operations will not work in this state. Please install CUDA toolkit version (12, 4) to continue using cuDF.
  warnings.warn(


In [2]:
import geopandas as gpd
from shapely.geometry import Polygon
from shapely import MultiPolygon
import os
from tqdm import tqdm

In [15]:
import h5py
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
from shapely import MultiPolygon
import os
from tqdm import tqdm


def read_boundary_hdf5(folder):
    all_boundaries = {}
    boundaries = None
    for i in tqdm(os.listdir(folder + '/cell_boundaries/')):
        with h5py.File(folder + '/cell_boundaries/' + i, "r") as f:
            for key in f['featuredata'].keys():
                if boundaries is not None:
                    boundaries.loc[key] = MultiPolygon([Polygon(f['featuredata'][key]['zIndex_3']['p_0']['coordinates'][()][0])]) # doesn't matter which zIndex we use, MultiPolygon to work with read function in spatialdata-io
                else:
                    boundaries = gpd.GeoDataFrame(index=[key], geometry=MultiPolygon([Polygon(f['featuredata'][key]['zIndex_3']['p_0']['coordinates'][()][0])]))
            all_boundaries[i] = boundaries
            boundaries = None
    all_concat = pd.concat(list(all_boundaries.values()))
    all_concat = all_concat[~all_concat.index.duplicated(keep='first')] # hdf5 can contain duplicates with same cell_id and position, removing those
    all_concat.rename_geometry('geometry_renamed', inplace=True)  # renaming to make it compatible with spatialdata-io
    all_concat["EntityID"] = all_concat.index  # renaming to make it compatible with spatialdata-io
    all_concat['ZIndex'] = 0  # adding to make it compatible with spatialdata-io
    all_concat.to_parquet(folder + '/cell_boundaries.parquet')
    
    count_path = f"{folder}/cell_by_gene.csv"
    obs_path = f"{folder}/cell_metadata.csv"

    data = pd.read_csv(count_path, index_col=0)
    obs = pd.read_csv(obs_path, index_col=0)

    data.index = obs.index.astype(str) # data index in old software is range(n_obs)
    data.index.name = "cell" # renaming to make it compatible with spatialdata-io
    obs.index.name = 'EntityID' # renaming to make it compatible with spatialdata-io
    data.to_csv(count_path)
    obs.to_csv(obs_path)

In [ ]:
read_boundary_hdf5('/external_data/other/Vizgen_redownload/HumanLiverCancerPatient1/')
read_boundary_hdf5('/external_data/other/Vizgen_redownload/HumanLiverCancerPatient2/')

In [ ]:
liver_vizgen_2 = spatialdata_io.merscope('/external_data/other/Vizgen_redownload/HumanLiverCancerPatient2/')


In [ ]:
liver_vizgen_1 = spatialdata_io.merscope('/external_data/other/Vizgen_redownload/HumanLiverCancerPatient1/')

## ProSeg segmentation

In [ ]:
%%bash

proseg detected_transcripts_cellpose.csv --merscope   

## Baysor Segmentation

In [4]:
import pandas as pd

In [5]:
tx_file = pd.read_csv('/external_data/other/Vizgen_redownload/HumanLiverCancerPatient2/cellpose_triple_zlayer/detected_transcripts.csv')

In [6]:
tx_file['fov_coarse'] = tx_file['fov'] // 10

In [7]:
fov_list = tx_file['fov_coarse'].unique()

In [8]:
tx_file.head()

,Unnamed: 0,barcode_id,global_x,global_y,global_z,x,y,fov,gene,transcript_id,cell_id,fov_coarse
0,107,0,6011.7544,17.578049,0.0,104.39474,162.75972,0,PDK4,ENST00000005178,2586372800021100015,0
1,253,0,6043.4150,29.064754,0.0,397.54468,269.11810,0,PDK4,ENST00000005178,2586372800021100023,0
2,447,0,6022.8360,41.203010,0.0,207.00000,381.50937,0,PDK4,ENST00000005178,2586372800021100031,0
3,654,0,6041.3940,53.028000,0.0,378.83395,491.00000,0,PDK4,ENST00000005178,2586372800021100042,0
4,1248,0,6077.3930,76.299280,0.0,712.15625,706.47473,0,PDK4,ENST00000005178,2586372800021100059,0


In [9]:
tx_file['cellID'] = tx_file['cell_id'].astype('category')
tx_file['cellID'] = tx_file['cellID'].cat.codes

In [ ]:
for i in tx_file['fov_coarse'].unique():
    tx_file[tx_file['fov_coarse']==i].to_csv(f'/external_data/other/Vizgen_redownload/HumanLiverCancerPatient2/cellpose_triple_zlayer/baysor_segmentation/liver_cancer_patient2_fov_{i}.csv')

In [20]:
for i in tx_file['fov_coarse'].unique():
    tx_file[tx_file['fov_coarse']==i].to_csv(f'/external_data/other/Vizgen_redownload/HumanLiverCancerPatient1/cellpose_triple_zlayer/baysor_segmentation/liver_cancer_patient1_fov_{i}.csv')

In [12]:
' '.join(fov_list.astype(str))

'0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251'

In [ ]:
%%bash

cd /external_data/other/Vizgen_redownload/HumanLiverCancerPatient2/baysor_segmentation
N=5
docker run -it -v /home/cane:/home/cane --rm vpetukhov/baysor:master
(
for VARIABLE in 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250; do
    ((i=i%N)); ((i++==0)) && wait
    baysor run -x global_x -y global_y -z global_z --gene gene liver_cancer_patient2_fov_$VARIABLE.csv  :cellID -o segmentation_m50_fov_$VARIABLE.csv -c config.toml --save-polygons=geojson --plot &  done
)

In [ ]:
%%bash

cd /external_data/other/Vizgen_redownload/HumanLiverCancerPatient1/baysor
N=5
docker run -it -v /home/cane:/home/cane --rm vpetukhov/baysor:master
(
for VARIABLE in 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251; do
    ((i=i%N)); ((i++==0)) && wait
    baysor run -x global_x -y global_y -z global_z --gene gene liver_cancer_patient1_fov_$VARIABLE.csv :cellID -o segmentation_m50_fov_$VARIABLE.csv -c config.toml --save-polygons=geojson --plot &  done
)

## Vizgen postprocessing tool segmentation

We used as input json files the following all download from the original repo

/data/yosef3/users/can/Vizgen_resegmentation/vizgen-postprocessing/example_analysis_algorithm/cellpose_default_3_ZLevel_nuclei_only.json
/data/yosef3/users/can/Vizgen_resegmentation/vizgen-postprocessing/example_analysis_algorithm/cellpose_default_1_ZLevel.json
/data/yosef3/users/can/Vizgen_resegmentation/vizgen-postprocessing/example_analysis_algorithm/cellpose_default_3_ZLevel.json

In [ ]:
%%bash

vpt --verbose --processes 4 run-segmentation \
--segmentation-algorithm /data/yosef3/users/can/Vizgen_resegmentation/vizgen-postprocessing/example_analysis_algorithm/cellpose_default_3_ZLevel_nuclei_only.json \
--input-images="images/mosaic_(?P<stain>[\w|-]+)_z(?P<z>[0-9]+).tif" \
--input-micron-to-mosaic images/micron_to_mosaic_pixel_transform.csv \
--output-path resegmentation_vizgen_patient1 \
--tile-size 2400 \
--tile-overlap 200